<a href="https://colab.research.google.com/github/DuplamenteH/spark-study/blob/main/pyspark_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

instalação do pyspark no colab


para instalar o pyspark primeiro teremos que instalar  -> java , spark
configurar java_home, e spark_home, instalar o findspark

instalação de configuração das variaveis de ambiente

In [ ]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
 
# tornar o pyspark "importável"
#import findspark
findspark.init('spark-3.0.1-bin-hadoop2.7')

Depois de importado e configurado vamos dar inicio ao spark

importaremos sparkSession para inicio a nossa sessão

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Data_Wrangling").getOrCreate()

In [ ]:
!wget -q https://raw.githubusercontent.com/Apress/applied-data-science-using-pyspark/main/Ch02/Chapter2_Data/movie_data_part1.csv

In [ ]:
dados = 'movie_data_part1.csv'
file_type = "csv"
infer_schema = "False"
first_row_is_header = "True"
delimiter  = "|"

primeira forma de carregar os dados

In [ ]:
df = spark.read.format(file_type)\
.option("inferSchema",infer_schema)\
.option("header",first_row_is_header)\
.option("sep",delimiter)\
.load(dados)

    

Lendo os dados de uma tabela hive


df2 = spark.sql("select * from database.table_name")

printamos os schema, assim podemos ver as colunas, se existem valores faltantes, e o tipo de cada coluna

In [ ]:
df.printSchema()

com a função df.dtypes, retornamos uma lista de tuplas com o nome da coluna e o tipo da coluna apenas

In [ ]:
df.dtypes

agr vamos saber o tamanho desse dataset, quantas linha tem o mesmo

In [ ]:
df.count()

uma maneira mai bonita seria assim:

In [ ]:
print("O total dos nossos registros é de  : {}".format(df.count()))

Coletando subcolunas para ter uma visão geral dos dados

In [ ]:
select_columns = ['id', 'budget', 'popularity', 'release_date',
                  'revenue', 'title']

In [ ]:
df =df.select(*select_columns)
df.show(30,False)

Valores faltantes do nosso dataset

algumas maneiras de calcular os valores faltantes

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df.filter((df['popularity']=='')|df['popularity'].isNull()|isnan(df['popularity'])).count()

na operação usando o condicional OR -> |,

na primeira condição achamos as strings nulas presentes em popularity comparando com strings vazias, na segunda condição chamamos a função .isNull() para a colunar popularity que retorna true quando alguma linha popularity é verdadeira e false se o contrario acontecer, na terceira condição usando a função isnan()-> not a number.

A segunda forma de achar a quantidade de valores faltantes é da seguinte forma.

In [ ]:
df.select([count(when((col(c)=='')| col(c).isNull()|isnan(c),c)).
           alias(c) for c in df.columns]).show()

#Pagina 64 // One-Way-Frequencies